In [169]:
import ast
from functools import partial
import networkx as nx
import numpy as np
from typing import Dict
import matplotlib.pyplot as plt
from dataclasses import dataclass
from mpi4py import MPI
from repast4py import context as ctx
from repast4py import core, random, schedule, logging, parameters
from repast4py.network import write_network, read_network

In [170]:
model = None


class RumorAgent(core.Agent):

    def __init__(self, nid: int, agent_type: int, rank: int, received_rumor=False, shadow=None):
        super().__init__(nid, agent_type, rank)
        self.received_rumor = received_rumor
        self.shadow = shadow or {}

    
def create_rumor_agent(nid, agent_type, rank, **kwargs):
    shadow = kwargs.get('shadow', {})
    return RumorAgent(nid, agent_type, rank, received_rumor=None, shadow=shadow)

def restore_agent(agent_data):
    uid = agent_data[0]
    received_rumor = agent_data[1]
    shadow = agent_data[2] if len(agent_data) > 2 else {}  # Handle cases where shadow might not be saved
    return RumorAgent(uid[0], uid[1], uid[2], received_rumor, shadow)

In [171]:
class Model:
    def __init__(self, comm, params):
        self.runner = schedule.init_schedule_runner(comm)
        self.runner.schedule_stop(params['stop.at'])

        fpath = params['network_file']
        self.context = ctx.SharedContext(comm)
        read_network(fpath, self.context, create_rumor_agent, restore_agent)
        self.net = self.context.get_projection('rumor_network')





    
    def start(self):
        self.runner.execute()

In [172]:
def run(params: Dict):
    global model
    model = Model(MPI.COMM_WORLD, params)
    model.start()


if __name__ == "__main__":
    params = {
        'network_file': 'layer1.txt',
        'initial_rumor_count': 5,
        'stop.at': 5,
        'rumor_probability': 0.1,
        'counts_file': 'output/rumor_counts.csv'
    }
    run(params)

ValueError: Error reading graph description file on line 2: "0 0 2 {"shadow":"{\\"key\\":\\"value\\"}"}". Expected node description with format: "node_id agent_type rank" following by an optional json agent attribute dictionary